In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sqlalchemy import create_engine

In [17]:
# csv path
kiva_loans_path="Data Sets/kiva_loans_clean"

In [18]:

kiva_loans_df=pd.read_csv(kiva_loans_path)

# Notes for Grader
The code in the next two cells was used in the shrink_csv.csv filed to make our CSV file smaller in order to add the file to Github.

In [19]:
# only keep columns with pertinant data
# kiva_loans_clean_df = kiva_loans_df[['LOAN_ID', 'LOAN_NAME',
#        'FUNDED_AMOUNT', 'LOAN_AMOUNT', 'STATUS', 'ACTIVITY_NAME', 'SECTOR_NAME',
#        'COUNTRY_CODE', 'COUNTRY_NAME', 'TOWN_NAME', 'CURRENCY',
#        'DISBURSE_TIME', 'LENDER_TERM', 'NUM_LENDERS_TOTAL',
#        'BORROWER_GENDERS', 'REPAYMENT_INTERVAL']]

In [20]:
# get data for only the four countries 

# kiva_country =  kiva_loans_df[(kiva_loans_df['COUNTRY_CODE'] == 'CR')|(kiva_loans_df['COUNTRY_CODE'] == 'SV')|(kiva_loans_df['COUNTRY_CODE'] == 'HN')|(kiva_loans_df['COUNTRY_CODE'] == 'NI')]

In [21]:
# drop no info 
kiva_country_clean = kiva_loans_df.dropna()

In [22]:
kiva_country_clean = kiva_country_clean.drop(columns='COUNTRY_CODE')

In [23]:
kiva_country_clean = kiva_country_clean.drop(columns='Unnamed: 0')

In [24]:
# check what date the data begins
kiva_country_clean.sort_values(by=['DISBURSE_TIME'])

,LOAN_ID,LOAN_NAME,FUNDED_AMOUNT,LOAN_AMOUNT,STATUS,ACTIVITY_NAME,SECTOR_NAME,COUNTRY_NAME,TOWN_NAME,CURRENCY,DISBURSE_TIME,LENDER_TERM,NUM_LENDERS_TOTAL,BORROWER_GENDERS,REPAYMENT_INTERVAL
149321,206,Eli Oved Zepeda,400.0,400.0,funded,Shoe Sales,Retail,Honduras,Siguatepeque,USD,2005-04-14 05:27:55.000 +0000,12.0,8,female,monthly
160762,244,Pastor David Suarez,250.0,250.0,funded,Pharmacy,Health,Honduras,Danli,USD,2005-04-14 05:27:55.000 +0000,12.0,5,male,monthly
127390,155,Maria Elena,200.0,200.0,funded,Tailoring,Services,Nicaragua,Managua,USD,2005-04-14 05:27:55.000 +0000,4.0,1,female,monthly
56939,220,Alba,350.0,350.0,funded,Clothing Sales,Clothing,Honduras,Danli,USD,2005-04-14 05:27:55.000 +0000,12.0,2,female,monthly
26485,236,Ramona Suyapa Zeron,175.0,175.0,funded,Bakery,Food,Honduras,Danli,USD,2005-04-14 05:27:55.000 +0000,12.0,1,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47044,1962779,Hector Jose,450.0,450.0,funded,Farming,Agriculture,Nicaragua,Wiwili,USD,2020-05-22 07:00:00.000 +0000,14.0,17,male,bullet
58645,1962338,Antonio Orfilio,1000.0,1000.0,funded,Farm Supplies,Agriculture,Nicaragua,Nueva Segovia,USD,2020-05-22 07:00:00.000 +0000,14.0,37,male,bullet
91007,1962340,Roger Omar,800.0,800.0,funded,Farm Supplies,Agriculture,Nicaragua,Nueva Segovia,USD,2020-05-22 07:00:00.000 +0000,14.0,5,male,bullet
148204,1962608,Corina,750.0,750.0,funded,Farming,Agriculture,Nicaragua,Matagalpa,USD,2020-05-23 07:00:00.000 +0000,14.0,2,female,bullet


In [25]:
# rename columns
kiva_country_clean = kiva_country_clean.rename(columns={
    'LOAN_ID': 'loan_id',
    'LOAN_NAME': 'loan_name',
    'FUNDED_AMOUNT':'amount_funded',
    'LOAN_AMOUNT':'loan_amount',
    'STATUS': 'loan_status',
    'ACTIVITY_NAME': 'activity_name',
    'SECTOR_NAME':'sector_name',
    'COUNTRY_NAME':'country_name',
    'TOWN_NAME':'town_name',
    'CURRENCY': 'currency',
    'DISBURSE_TIME':'dispursement_date',
    'LENDER_TERM':'length_of_loan',
    'NUM_LENDERS_TOTAL':'total_number_of_lenders',
    'BORROWER_GENDERS':'borrower_gender',
    'REPAYMENT_INTERVAL':'repayment_interval'
})

In [26]:
kiva_country_clean_df = kiva_country_clean[(kiva_country_clean['borrower_gender'] == 'male')|(kiva_country_clean['borrower_gender'] == 'female')]

In [27]:
# connect to SQL
rds_connection_string = "admin1:12345@localhost:5432/ETL_Gary_and_Maria"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [28]:
engine.table_names()

['kiva_data', 'worldbank_data']

In [29]:
# Export DF to SQL table
kiva_country_clean_df.to_sql(name='kiva_data', con=engine, if_exists='append', index=False)

In [30]:
pd.read_sql_query('select * from kiva_data', con=engine).head()

,loan_id,loan_name,amount_funded,loan_amount,loan_status,activity_name,sector_name,country_name,town_name,currency,dispursement_date,length_of_loan,total_number_of_lenders,borrower_gender,repayment_interval
0,419930,Wilfredo,350.0,350.0,funded,Clothing Sales,Clothing,El Salvador,"San Francisco Gotera, Morazan",USD,2012-04-03 07:00:00.000 +0000,14.0,7,male,monthly
1,923021,Denis Noe Antonio,650.0,1000.0,expired,Livestock,Agriculture,El Salvador,Sensuntepeque,USD,2015-06-26 07:00:00.000 +0000,17.0,14,male,monthly
2,1616035,Reina Isabel,800.0,800.0,funded,Poultry,Agriculture,El Salvador,San Vicente,USD,2018-09-11 07:00:00.000 +0000,20.0,24,female,monthly
3,355527,Williams Aguistin,1025.0,1025.0,funded,Services,Services,Nicaragua,Leon,NIO,2011-10-31 07:00:00.000 +0000,21.0,36,male,monthly
4,868483,Blanca Lyliam,300.0,300.0,funded,Personal Medical Expenses,Health,El Salvador,El Transito,USD,2015-04-08 07:00:00.000 +0000,12.0,12,female,monthly
